<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [64]:
url = 'https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html'
page = requests.get(url).content
soup = BeautifulSoup(page)

In [65]:
table = []
for row in soup.find_all('tr'):
    data_row = []
    for value in row.find_all('td'):
        data_row.append(value.get_text())
    table.append(data_row)

In [66]:
df = pd.DataFrame([x for x in table if len(x) == 6])
df.columns = ['Year', 'Date', 'Deposit Facility', 'MFO Fixed', 'MFO Variable', 'Marginal lending facility']

In [67]:
df.Year = df.Year.str.strip().replace('', np.nan).ffill()

In [70]:
df['date'] = pd.to_datetime(df.Date.str.split('.').str[0] + ' ' + df.Year)

In [71]:
df.drop(columns=['Year', 'Date']).set_index('date').replace('-', np.nan).to_csv('ECB_primary_rates.csv')

In [47]:
df = df.drop(columns=['Year', 'Date']).set_index('date').loc['1999':'2013'].replace('-', np.nan)

In [49]:
df['MFO'] = df['MFO Fixed'].fillna(df['MFO Variable'])

In [59]:
df['MFO'].apply(float).sort_index().diff().groupby(lambda x: x.year).value_counts().unstack(1).fillna(0).applymap(int)

MFO,-0.75,-0.50,-0.25,0.00,0.25,0.50
date,,,,,,
1999,0,1,0,2,0,1
2000,0,0,0,1,5,1
2001,0,2,2,0,0,0
2002,0,1,0,0,0,0
2003,0,1,1,0,0,0
2005,0,0,0,0,1,0
2006,0,0,0,0,5,0
2007,0,0,0,0,2,0
2008,1,1,0,0,1,0
